In [1]:
import tweepy
import json
from requests.exceptions import Timeout, ConnectionError
import time
from time import sleep
import configparser
import ssl
import pandas as pd
import pyodbc


In [2]:

import pickle
from scipy.stats import stats
import spacy
from tqdm import *
import re

In [3]:

# Setting the connection strings
server = 'tcp:isye-6420-project.database.windows.net,1433' 
database = 'topic_modelling' 
username = 'project_administrator' 
password = 'isye_6420_admin' 
conn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
#SQLite cursor
c = conn.cursor()


In [5]:

# Method to read the twitter messages into the pandas dataframe
def read_sql():
    train_data =  pd.read_sql_query('SELECT  [tweet_text],[final_normalized_topic] FROM topic_modelling_ensemble_V4', conn) 
    return(train_data)


In [6]:
#train_data=read_sql()


In [7]:
print(train_data)

                                              tweet_text  \
0      In 2016, there wasn't a single question on cli...   
1      A leader among the next generation of Democrat...   
2      A moving tribute from Mayor @KeishaBottoms for...   
3      Let's never forget that the Trump administrati...   
4      Incredibly powerful words from @GabbyGiffords ...   
...                                                  ...   
53696  “7 in 10 Americans say the economy is in good ...   
53697  #MedicareForAll costs $32 trillion, imposes a ...   
53698  SC is enjoying lower gas prices than during th...   
53699  -Over 7 in 10 Americans feel positively about ...   
53700  Met with SSG Amanda Kelley, an Easley native, ...   

         final_normalized_topic  
0      job economy unemployment  
1               veteran service  
2            vote election mail  
3               veteran service  
4                  gun violence  
...                         ...  
53696  job economy unemployment  
53697  

In [8]:
# Method to read the new harnessed twitter messages into the pandas dataframe
def read_sql1():
    test_data_1 =  pd.read_sql_query('select [tweet_text],[political_party],[time],[user] from tweets_test', conn) 
    return(test_data_1)

In [9]:
# Step 1 : Data Loading 
    
# Executing method to read twitter messages into dataframe
test_data_1 = read_sql1()

In [10]:
print(test_data_1)

                                             tweet_text   political_party  \
0     Our men and women in uniform make great sacrif...  Democratic Party   
1                          COUNT EVERY MILITARY BALLOT!  Republican Party   
2     #ICYMI An important move for ensuring the rule...  Republican Party   
3     I know 75 million Americans willing to help yo...  Democratic Party   
4                               https://t.co/ZhdVxJKv77  Democratic Party   
...                                                 ...               ...   
7663  Don’t let the line scare you...it moves quick!...  Democratic Party   
7664  We need 60% of the population to have #COVID19...  Democratic Party   
7665  You can vote early this week! Check out the lo...  Democratic Party   
7666  7 more days left to vote!\n\nhttps://t.co/g89Z...  Democratic Party   
7667  My statement on the U.S. Senate’s confirmation...  Republican Party   

                     time             user  
0     2020-11-05 21:00:00     

In [11]:
# Method to read the new harnessed twitter messages into the pandas dataframe
def read_sql11():
    test_data =  pd.read_sql_query('select [tweet_text] from tweets_test', conn) 
    return(test_data)
# Step 1 : Data Loading 
    
# Executing method to read twitter messages into dataframe
test_data = read_sql11()
print(test_data)

                                             tweet_text
0     Our men and women in uniform make great sacrif...
1                          COUNT EVERY MILITARY BALLOT!
2     #ICYMI An important move for ensuring the rule...
3     I know 75 million Americans willing to help yo...
4                               https://t.co/ZhdVxJKv77
...                                                 ...
7663  Don’t let the line scare you...it moves quick!...
7664  We need 60% of the population to have #COVID19...
7665  You can vote early this week! Check out the lo...
7666  7 more days left to vote!\n\nhttps://t.co/g89Z...
7667  My statement on the U.S. Senate’s confirmation...

[7668 rows x 1 columns]


In [12]:
# Step 2: Data Cleaning 

   
# print the train_data column names 
print(test_data.columns)

# Load the regular expression library
import re

#  Remove URLs
train_data['tweet_text'] = train_data['tweet_text'].apply(lambda x: re.split('https:\/\/.*', str(x))[0])

# Remove punctuation
train_data['tweet_text'] = train_data['tweet_text'].map(lambda x: re.sub('[,\.!?@]', '', x))

# Convert the titles to lowercase
train_data['tweet_text'] = train_data['tweet_text'].map(lambda x: x.lower())

# Removing the word 'amp' 
train_data['tweet_text'] = train_data['tweet_text'].map(lambda x: re.sub('amp', '', x))

# Removing Stopwords 
#import nltk 
#nltk.download('stopwords')  #First time after you have installed nltk
#from nltk.corpus import stopwords
#STOPWORDS = stopwords.words('english')
#train_data['tweet_text'] = train_data['tweet_text'].apply(lambda x: [item for item in x if item not in STOPWORDS])

print(train_data['tweet_text'])

Index(['tweet_text'], dtype='object')
0        in 2016 there wasn't a single question on clim...
1        a leader among the next generation of democrat...
2        a moving tribute from mayor keishabottoms for ...
3        let's never forget that the trump administrati...
4        incredibly powerful words from gabbygiffords -...
                               ...                        
53696    “7 in 10 americans say the economy is in good ...
53697    #medicareforall costs $32 trillion imposes a g...
53698    sc is enjoying lower gas prices than during th...
53699    -over 7 in 10 americans feel positively about ...
53700    met with ssg amanda kelley an easley native in...
Name: tweet_text, Length: 53701, dtype: object


In [13]:
       
# 5. Removing Stopwords 

import nltk 
from nltk.corpus import stopwords

STOPWORDS = set(stopwords.words('english','spanish'))
def remove(text,wordset=STOPWORDS):
    splits = text.split()
    result=[]
    for split in splits:
        if split not in wordset:
            result.append(split)
    return (" ".join(result))  
train_data['tweet_text'] = train_data['tweet_text'].apply(lambda x: remove(x))

print(train_data['tweet_text'].head(1000))

0      2016 single question climate change presidenti...
1      leader among next generation democrats - commi...
2      moving tribute mayor keishabottoms dear friend...
3      let's never forget trump administration extrem...
4      incredibly powerful words gabbygiffords - turn...
                             ...                        
995    thank youth conservation corps (npsyouth) team...
996    trump admin’s decision end #2020census month e...
997    see stronger economy bringing people disabilit...
998    thank rep jimlangevin partnering bipartisan le...
999    congratulations abackyardfarm’s joan james col...
Name: tweet_text, Length: 1000, dtype: object


In [14]:

#  6. To retain only the Nouns and Adjectives in every tweet 

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ']):
    """https://spacy.io/api/annotation"""
    doc = nlp(texts) 
    texts_out = " ".join([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out     

# Do lemmatization keeping only noun, adj parts of speech (POS)
train_data['tweet_text'] = train_data['tweet_text'].apply(lambda x: lemmatization(x, allowed_postags=['NOUN', 'ADJ']))


In [15]:
# 7. To eliminate additional words 


eliminate_words = ['today', 'year', 'people', 'time', 'great', 'day', 'morning', 'evening', 'community', 'important',
                   'many', 'family', 'sure', 'visit', 'https', 'new', 'work', 'proud', 'thank', 'good', 'way', 'help',
                   'need', 'member', 'week', 'nation', 'support', 'last', 'effort', 'yesterday', 'news', 'watch', 'state',
                   'safe', 'happy', 'birthday', 'colleague', 'tonight', 'part', 'much', 'now', 'country', 'leader', 'program',
                   'congratulation', 'conversation', 'fact', 'government', 'house', 'right', 'tomorrow', 'question', 'long',
                   'public', 'big', 'million', 'benefit', 'information', 'report' , 'friend', 'life', 'bad' , 'word', 'true',
                   'woman', 'man', 'local', 'action', 'resource', 'high','school', 'opportunity', 'town', 'event', 'meeting',
                   'tune', 'pm', 'student', 'live', 'discussion', 'discuss', 'child', 'world', 'policy', 'condition', 'hand',  
                   'office', 'place', 'resident', 'site' , 'concern', 'city', 'free', 'congressional', 'WORKER', 'history',
                   'M', 'P', 'wonderful', 'team', 'update', 'facebook', 'minute', 'op', 'water', 'food', 'hour', 'pm', 
                   'thing', 'art', 'folk', 'annual', 'prayer', 'thought', 'heart', 'statement', 'story', 'special', 'pre',
                   'honor', 'young', 'neighbor', 'night', 'district', 'late', 'moment', 'case', 'census', 'person', 'voice', 
                   'wrong',  'share', 'victory', 'grateful', 'century', 'kid', 'difficult', 'afternoon', 'love', 'youth', 
                   'fellow', 'mother',  'comment', 'official','p','m', 'crisis', 'issue']

train_data['tweet_text'] = train_data['tweet_text'].apply(lambda x: remove(x, wordset=eliminate_words ))
print(train_data)

                                              tweet_text  \
0      single climate change presidential debate lett...   
1      next generation service future first step well...   
2                    mayor dear vote text vote plan vote   
3      administration extreme immigration spouse mili...   
4      powerful gabbygifford tragedy purpose end gun ...   
...                                                  ...   
53696                        economy shape quarter potus   
53697  medicareforall size health plan taxis choice l...   
53698  low gas price obama republican above energy pr...   
53699                    job market optimism workforce %   
53700                                          service #   

         final_normalized_topic  
0      job economy unemployment  
1               veteran service  
2            vote election mail  
3               veteran service  
4                  gun violence  
...                         ...  
53696  job economy unemployment  
53697  

In [16]:
def replace_synonyms(text):
    splits = text.split()
    result=[]
    for split in splits:
        if split in synonyms:
            result.append(synonyms[split])
        else:
            result.append(split)
    return (" ".join(result))        

#replace synonyms
synonyms = {'donald':'trump','realdonaldtrump':'trump','joebiden':'biden'}
synonyms = {'paycheck protection':'paycheck protection program','protection program':'paycheck protection program'}
synonyms = {'admin':'administration'}
synonyms = {'realdonaldtrump':'trump'}
train_data['tweet_text'] = train_data['tweet_text'].apply(lambda x: replace_synonyms(x))
print(train_data)

                                              tweet_text  \
0      single climate change presidential debate lett...   
1      next generation service future first step well...   
2                    mayor dear vote text vote plan vote   
3      administration extreme immigration spouse mili...   
4      powerful gabbygifford tragedy purpose end gun ...   
...                                                  ...   
53696                        economy shape quarter potus   
53697  medicareforall size health plan taxis choice l...   
53698  low gas price obama republican above energy pr...   
53699                    job market optimism workforce %   
53700                                          service #   

         final_normalized_topic  
0      job economy unemployment  
1               veteran service  
2            vote election mail  
3               veteran service  
4                  gun violence  
...                         ...  
53696  job economy unemployment  
53697  

In [17]:
x_training=train_data['tweet_text']
print(x_training)

0        single climate change presidential debate lett...
1        next generation service future first step well...
2                      mayor dear vote text vote plan vote
3        administration extreme immigration spouse mili...
4        powerful gabbygifford tragedy purpose end gun ...
                               ...                        
53696                          economy shape quarter potus
53697    medicareforall size health plan taxis choice l...
53698    low gas price obama republican above energy pr...
53699                      job market optimism workforce %
53700                                            service #
Name: tweet_text, Length: 53701, dtype: object


In [20]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
#from self import self

count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(x_training)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

# after trainning the data
#vector = CountVectorizer()
#vector.fit(self.x_training)
#X_train = vector.transform(self.x_training)


# use vector.vocabulary for predict
#vector = CountVectorizer(vocabulary=vector.vocabulary_) #vocabulary is a parameter, it should be vocabulary_ as it is an attribute.
#text_vector = vector.transform(text)

In [21]:
print(X_train_tfidf)

  (0, 15287)	0.2105767306024062
  (0, 12813)	0.23477427584072535
  (0, 10537)	0.33802822791627035
  (0, 9419)	0.18072655874507373
  (0, 6464)	0.2344508749888831
  (0, 4250)	0.6959232415236806
  (0, 3054)	0.3318602165094053
  (0, 2704)	0.32280912913154225
  (1, 18407)	0.24514690495452202
  (1, 15864)	0.24077157939710758
  (1, 15016)	0.18135759019158348
  (1, 11177)	0.24123776487061196
  (1, 8935)	0.4159307167945514
  (1, 6904)	0.24817601075890938
  (1, 6787)	0.4331113275844159
  (1, 6312)	0.227681098576863
  (1, 4434)	0.31468256729049354
  (1, 4421)	0.4666978577719204
  (2, 18169)	0.6598297025127012
  (2, 16624)	0.4107965121454525
  (2, 12456)	0.2702186570340643
  (2, 10064)	0.4151807607333811
  (2, 4242)	0.38791468469070484
  (3, 18024)	0.2552379080459617
  (3, 15684)	0.44989025353661205
  :	:
  (53697, 9885)	0.32990852145478033
  (53697, 9411)	0.26919016631226445
  (53697, 9327)	0.2758348024006175
  (53697, 7602)	0.14386417487524417
  (53697, 5568)	0.2601351958182377
  (53697, 4416)	0

In [23]:
Y_train=train_data['final_normalized_topic']
print(Y_train)

0        job economy unemployment
1                 veteran service
2              vote election mail
3                 veteran service
4                    gun violence
                   ...           
53696    job economy unemployment
53697                 health care
53698           drug prescription
53699    job economy unemployment
53700             veteran service
Name: final_normalized_topic, Length: 53701, dtype: object


In [24]:
import nltk

clf = MultinomialNB().fit(X_train_tfidf, Y_train)
print(clf)

MultinomialNB()


In [25]:
# Step 2: Data Cleaning 

   
# print the test_data column names 
print(test_data.columns)

# Load the regular expression library
import re

#  Remove URLs
test_data['tweet_text'] = test_data['tweet_text'].apply(lambda x: re.split('https:\/\/.*', str(x))[0])

# Remove punctuation
test_data['tweet_text'] = test_data['tweet_text'].map(lambda x: re.sub('[,\.!?@]', '', x))

# Convert the titles to lowercase
test_data['tweet_text'] = test_data['tweet_text'].map(lambda x: x.lower())

# Removing the word 'amp' 
test_data['tweet_text'] = test_data['tweet_text'].map(lambda x: re.sub('amp', '', x))

# Removing Stopwords 
#import nltk 
#nltk.download('stopwords')  #First time after you have installed nltk
#from nltk.corpus import stopwords
#STOPWORDS = stopwords.words('english')
#train_data['tweet_text'] = train_data['tweet_text'].apply(lambda x: [item for item in x if item not in STOPWORDS])

print(test_data['tweet_text'])

Index(['tweet_text'], dtype='object')
0       our men and women in uniform make great sacrif...
1                             count every military ballot
2       #icymi an important move for ensuring the rule...
3       i know 75 million americans willing to help yo...
4                                                        
                              ...                        
7663    don’t let the line scare youit moves quick \n\...
7664    we need 60% of the population to have #covid19...
7665    you can vote early this week check out the loc...
7666                         7 more days left to vote\n\n
7667    my statement on the us senate’s confirmation o...
Name: tweet_text, Length: 7668, dtype: object


In [26]:
       
# 5. Removing Stopwords 

import nltk 
from nltk.corpus import stopwords

STOPWORDS = set(stopwords.words('english','spanish'))
def remove(text,wordset=STOPWORDS):
    splits = text.split()
    result=[]
    for split in splits:
        if split not in wordset:
            result.append(split)
    return (" ".join(result))  
test_data['tweet_text'] = test_data['tweet_text'].apply(lambda x: remove(x))

print(test_data['tweet_text'])

0       men women uniform make great sacrifices protec...
1                             count every military ballot
2       #icymi important move ensuring rules honestly ...
3             know 75 million americans willing help pack
4                                                        
                              ...                        
7663    don’t let line scare youit moves quick can’t w...
7664    need 60% population #covid19 antibodies reach ...
7665    vote early week check locations times various ...
7666                                     7 days left vote
7667    statement us senate’s confirmation judge amy c...
Name: tweet_text, Length: 7668, dtype: object


In [27]:

#  6. To retain only the Nouns and Adjectives in every tweet 

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ']):
    """https://spacy.io/api/annotation"""
    doc = nlp(texts) 
    texts_out = " ".join([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out     

# Do lemmatization keeping only noun, adj parts of speech (POS)
test_data['tweet_text'] = test_data['tweet_text'].apply(lambda x: lemmatization(x, allowed_postags=['NOUN', 'ADJ']))


In [28]:
# 7. To eliminate additional words 


eliminate_words = ['today', 'year', 'people', 'time', 'great', 'day', 'morning', 'evening', 'community', 'important',
                   'many', 'family', 'sure', 'visit', 'https', 'new', 'work', 'proud', 'thank', 'good', 'way', 'help',
                   'need', 'member', 'week', 'nation', 'support', 'last', 'effort', 'yesterday', 'news', 'watch', 'state',
                   'safe', 'happy', 'birthday', 'colleague', 'tonight', 'part', 'much', 'now', 'country', 'leader', 'program',
                   'congratulation', 'conversation', 'fact', 'government', 'house', 'right', 'tomorrow', 'question', 'long',
                   'public', 'big', 'million', 'benefit', 'information', 'report' , 'friend', 'life', 'bad' , 'word', 'true',
                   'woman', 'man', 'local', 'action', 'resource', 'high','school', 'opportunity', 'town', 'event', 'meeting',
                   'tune', 'pm', 'student', 'live', 'discussion', 'discuss', 'child', 'world', 'policy', 'condition', 'hand',  
                   'office', 'place', 'resident', 'site' , 'concern', 'city', 'free', 'congressional', 'WORKER', 'history',
                   'M', 'P', 'wonderful', 'team', 'update', 'facebook', 'minute', 'op', 'water', 'food', 'hour', 'pm', 
                   'thing', 'art', 'folk', 'annual', 'prayer', 'thought', 'heart', 'statement', 'story', 'special', 'pre',
                   'honor', 'young', 'neighbor', 'night', 'district', 'late', 'moment', 'case', 'census', 'person', 'voice', 
                   'wrong',  'share', 'victory', 'grateful', 'century', 'kid', 'difficult', 'afternoon', 'love', 'youth', 
                   'fellow', 'mother',  'comment', 'official','p','m', 'crisis', 'issue']

test_data['tweet_text'] = test_data['tweet_text'].apply(lambda x: remove(x, wordset=eliminate_words ))
print(test_data)

                                             tweet_text
0                     sacrifice virtual veteran p.m. pt
1                                       military ballot
2                                             move rule
3                                               willing
4                                                      
...                                                 ...
7663       line scare youit move quick # yourvotematter
7664  % population # covid19 antibody herd immunity ...
7665  early check location various early voting polling
7666                                               vote
7667             confirmation judge scotus acbconfirmed

[7668 rows x 1 columns]


In [29]:
def replace_synonyms(text):
    splits = text.split()
    result=[]
    for split in splits:
        if split in synonyms:
            result.append(synonyms[split])
        else:
            result.append(split)
    return (" ".join(result))        

#replace synonyms
synonyms = {'donald':'trump','realdonaldtrump':'trump','joebiden':'biden'}
synonyms = {'paycheck protection':'paycheck protection program','protection program':'paycheck protection program'}
synonyms = {'admin':'administration'}
synonyms = {'realdonaldtrump':'trump'}
test_data['tweet_text'] = test_data['tweet_text'].apply(lambda x: replace_synonyms(x))
print(test_data)

                                             tweet_text
0                     sacrifice virtual veteran p.m. pt
1                                       military ballot
2                                             move rule
3                                               willing
4                                                      
...                                                 ...
7663       line scare youit move quick # yourvotematter
7664  % population # covid19 antibody herd immunity ...
7665  early check location various early voting polling
7666                                               vote
7667             confirmation judge scotus acbconfirmed

[7668 rows x 1 columns]


In [30]:
x_test=test_data['tweet_text']

In [31]:

#count_vect = CountVectorizer()
X_test= count_vect.transform(x_test)


In [33]:
y_pred=clf.predict(X_test)

In [34]:
print(y_pred)

['veteran service' 'vote election mail' 'health care' ...
 'vote election mail' 'vote election mail' 'health care']


In [35]:
print(len(y_pred))

7668


In [36]:
#pd.options.display.float_format = '{:,.16f}'.format
df={"tweet":test_data_1["tweet_text"],"political_party":test_data_1["political_party"],'time':test_data_1["time"],'User':test_data_1['user'],"Predicted_topic":y_pred}
d = pd.DataFrame(data=df)
d

,tweet,political_party,time,User,Predicted_topic
0,Our men and women in uniform make great sacrif...,Democratic Party,2020-11-05 21:00:00,RepTedLieu,veteran service
1,COUNT EVERY MILITARY BALLOT!,Republican Party,2020-11-05 22:07:05,RepMattGaetz,vote election mail
2,#ICYMI An important move for ensuring the rule...,Republican Party,2020-11-07 13:23:40,WarrenDavidson,health care
3,I know 75 million Americans willing to help yo...,Democratic Party,2020-11-07 17:34:55,RepJoeKennedy,veteran service
4,https://t.co/ZhdVxJKv77,Democratic Party,2020-11-07 16:47:40,BillPascrell,job economy unemployment
...,...,...,...,...,...
7663,Don’t let the line scare you...it moves quick!...,Democratic Party,2020-10-28 13:47:26,RepBeatty,vote election mail
7664,We need 60% of the population to have #COVID19...,Democratic Party,2020-10-28 16:10:15,CongressmanRaja,covid19 testing
7665,You can vote early this week! Check out the lo...,Democratic Party,2020-10-27 18:02:16,CongressmanRaja,vote election mail
7666,7 more days left to vote!\n\nhttps://t.co/g89Z...,Democratic Party,2020-10-27 13:02:59,CongressmanRaja,vote election mail


In [39]:
d = d.rename(columns={"tweet": "tweet_text", "time": "tweet_time", "User": "tweet_user"})



In [41]:
import pyodbc
## Setting the connection strings
server = 'tcp:isye-6420-project.database.windows.net,1433' 
database = 'topic_modelling' 
username = 'project_administrator' 
password = 'isye_6420_admin' 
conn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
##SQLite cursor
c = conn.cursor()
## Creating the topic_modelling_ensemble Table 
temp_creator = '''CREATE TABLE topic_prediction (tweet_text nvarchar(max), political_party nvarchar(50) , tweet_time nvarchar(50), tweet_user nvarchar(50), Predicted_topic nvarchar(50));'''
c.execute(temp_creator)
c.commit()


In [42]:
import pandas as pd
import time
from urllib.parse import quote_plus
import pyodbc
from sqlalchemy import create_engine, MetaData, Table, select
server = 'tcp:isye-6420-project.database.windows.net,1433' 
database = 'topic_modelling' 
username = 'project_administrator' 
password = 'isye_6420_admin' 
conn ='DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password +';Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;'
quoted = quote_plus(conn)
engine=create_engine('mssql+pyodbc:///?odbc_connect={}'.format(quoted))
conn = engine.connect()
TableName = 'topic_prediction'
metadata = MetaData(conn)
start_time = time.time()

d.to_sql(TableName,conn, method='multi', index=False, if_exists='append', chunksize=100)
print("--- %s seconds ---" % (time.time() - start_time))

--- 16.367491483688354 seconds ---


In [44]:
clf

MultinomialNB()